# Baseline Classifier

In [1]:
import tensorflow as tf
from tensorflow import image as img
import os
import shutil, sys
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import keras
from keras.utils import plot_model
from keras.applications.mobilenet import MobileNet
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

Using TensorFlow backend.


In [3]:
from keras.models import load_model
model1 = load_model('keras_aug_4_15_12.h5')
#model2 = load_model('2clasifier_40_epochs.h5')

In [4]:
#Read the test images and translate the images data into a numpy array
test_dir = './test'
rsltn = 224
test_count = len(os.listdir(test_dir))
test_data = np.zeros(shape=[test_count, rsltn, rsltn, 3])
test_ids = []
curr_idx = 0

for each_file in os.listdir(test_dir):
    test_ids.append(each_file)
    pic = image.load_img(os.path.join(test_dir, each_file), target_size=(rsltn, rsltn))
    x = image.img_to_array(pic)
    x = x/255
    test_data[curr_idx, :] = x
    curr_idx += 1

In [5]:
val1_dir = './validation1'
batch_size = 30
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator_1 = val_datagen.flow_from_directory(
                        val1_dir,
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')
class_dict = val_generator_1.class_indices
reverse_class_dict = {}
for i in class_dict:
    reverse_class_dict[class_dict[i]] = i

Found 360 images belonging to 12 classes.


In [6]:
val_generator_1.class_indices

{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

In [7]:
reverse_class_dict

{0: 'Black-grass',
 1: 'Charlock',
 2: 'Cleavers',
 3: 'Common Chickweed',
 4: 'Common wheat',
 5: 'Fat Hen',
 6: 'Loose Silky-bent',
 7: 'Maize',
 8: 'Scentless Mayweed',
 9: 'Shepherds Purse',
 10: 'Small-flowered Cranesbill',
 11: 'Sugar beet'}

In [8]:
predictions1 = model1.predict(test_data)
test_count = predictions1.shape[0]
predicted_classes_model1= [reverse_class_dict[x] for x in np.argmax(predictions1, axis=1)]

In [9]:
predicted_classes_model1[2]

'Small-flowered Cranesbill'

# 2nd Classifier

In [10]:
model2 = load_model('2class_best_1.h5')

In [11]:
len(predicted_classes_model1)

794

In [13]:
count = 0;
img_size = 224
predicted_classes_final = []
for i in range(test_count):
    if(predicted_classes_model1[i] == 'Black-grass' or predicted_classes_model1[i] == 'Loose Silky-bent'):
        pic = image.load_img(os.path.join(test_dir, test_ids[i]), target_size=(img_size, img_size))
        x = image.img_to_array(pic)
        x = x/255
        predict2 = model2.predict(x.reshape(1, img_size,img_size,3))
        if(predict2[0,0] > predict2[0,1]):            
            predicted_classes_final.append('Black-grass')
        else:
            predicted_classes_final.append('Loose Silky-bent')
        count += 1
    else:
        predicted_classes_final.append(predicted_classes_model1[i])
        #print(predict2.shape)
        #print("pred1_BG=", predictions1[i, 0], "pred1_L=", predictions1[i, 6], "pred2_BG=", predict2[0,0], "pred2_L=", predict2[0,1])
    

In [26]:
len(predicted_classes_final)

794

In [27]:
import pandas as pd
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes_final})
submission.to_csv('ensemble_classifier_3.csv', encoding="utf8", index=False)
from IPython.display import FileLink
FileLink('ensemble_classifier_3.csv')

/home/marineni0777/ensemble_classifier_3.csv

# Confusion Matrix On Validation Data

In [20]:
test_dir = './Plant-Seedlings-Classification/validation'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predictions1 = model1.predict_generator(test_generator,steps = nb_samples)

Found 360 images belonging to 12 classes.


In [23]:
test_count = predictions1.shape[0]
predicted_classes_model1= np.argmax(predictions1, axis=1)
count = 0;
predicted_classes_final = []

for i in range(test_count):
    if(predicted_classes_model1[i] == 0 or predicted_classes_model1[i] == 6):
        #os.path.abspath(test_ids[i])
        pic = image.load_img(os.path.join(test_dir, filenames[i]), target_size=(img_size, img_size))
        x = image.img_to_array(pic)
        x = x/255
        predict2 = model2.predict(x.reshape(1, img_size,img_size,3))
        if(predict2[0,0] > predict2[0,1]):            
            predicted_classes_final.append(0)
        else:
            predicted_classes_final.append(6)
        count += 1
    else:
        predicted_classes_final.append(predicted_classes_model1[i])


In [28]:
predicted_classes_final = np.array(predicted_classes_final)

In [27]:
val_labels = np.zeros(360, dtype=int)
for i in range(12):
    for j in range(30):
        val_labels[i*30+j] = int(i);

In [30]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(val_labels, predicted_classes_final)

array([[25,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0],
       [ 0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 29,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 30,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 30,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 30,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1, 29,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 30,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1, 29,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 29,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 30]])